# External validation of BERT model  
1. External validation using fake news data from 2016
2. External validation from other fake news articles

In [1]:
!pip install transformers 

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import regex as re
import seaborn as sns
import torch
import torch.nn as nn

from datetime import date
from google.colab import drive 
from sklearn.metrics import classification_report
from torch.utils.data import TensorDataset, DataLoader
from transformers import AutoModel, BertTokenizerFast


# specify GPU
device = torch.device("cuda")


     |████████████████████████████████| 1.1MB 3.7MB/s 
     |████████████████████████████████| 890kB 45.5MB/s 
     |████████████████████████████████| 1.1MB 50.3MB/s 
     |████████████████████████████████| 3.0MB 50.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=c5532b47e06a4a3bb46c1bb137757105347fd71f9f9ab8fe06acd770cf61c926
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
drive.mount('/content/drive', force_remount=True)
path = '/content/drive/My Drive/DS4A-Project/'

Mounted at /content/drive


## 1. External validation with fake news from 2016

In [5]:
df_2016 = pd.read_csv(path +'Data/fakenews_2016/fake.csv')

In [6]:
df_2016.head()

,uuid,ord_in_thread,author,published,title,text,language,crawled,site_url,country,domain_rank,thread_title,spam_score,main_img_url,replies_count,participants_count,likes,comments,shares,type
0,6a175f46bcd24d39b3e962ad0f29936721db70db,0,Barracuda Brigade,2016-10-26T21:41:00.000+03:00,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,english,2016-10-27T01:49:27.168+03:00,100percentfedup.com,US,25689.0,Muslims BUSTED: They Stole Millions In Gov’t B...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
1,2bdc29d12605ef9cf3f09f9875040a7113be5d5b,0,reasoning with facts,2016-10-29T08:47:11.259+03:00,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...,english,2016-10-29T08:47:11.259+03:00,100percentfedup.com,US,25689.0,Re: Why Did Attorney General Loretta Lynch Ple...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
2,c70e149fdd53de5e61c29281100b9de0ed268bc3,0,Barracuda Brigade,2016-10-31T01:41:49.479+02:00,BREAKING: Weiner Cooperating With FBI On Hilla...,Red State : \nFox News Sunday reported this mo...,english,2016-10-31T01:41:49.479+02:00,100percentfedup.com,US,25689.0,BREAKING: Weiner Cooperating With FBI On Hilla...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
3,7cf7c15731ac2a116dd7f629bd57ea468ed70284,0,Fed Up,2016-11-01T05:22:00.000+02:00,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...,english,2016-11-01T15:46:26.304+02:00,100percentfedup.com,US,25689.0,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,0.068,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias
4,0206b54719c7e241ffe0ad4315b808290dbe6c0f,0,Fed Up,2016-11-01T21:56:00.000+02:00,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,english,2016-11-01T23:59:42.266+02:00,100percentfedup.com,US,25689.0,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,0.865,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias


In [7]:
df_2016 = df_2016[["title", "text", "published", "language", "country", "type"]]
df_2016["year"] = df_2016["published"].apply(lambda x: x[:4])
df_2016.drop(columns=["published"], inplace = True)
# select only those in English and US
df_2016 = df_2016[(df_2016["language"] == "english") & (df_2016["country"] == "US")]

In [8]:
df_2016.year.value_counts()

2016    10136
Name: year, dtype: int64

In [9]:
df_2016.dropna(inplace = True)

In [10]:
df_2016.type.value_counts()

bs            8485
conspiracy     299
hate           243
bias           237
junksci        100
satire         100
fake            18
state            7
Name: type, dtype: int64

In [11]:
# we'll use all types except "state"
df_2016 = df_2016[df_2016["type"] != "state"]

In [12]:
df_2016.reset_index(inplace = True, drop = True)

In [13]:
df_2016.head()

,title,text,language,country,type,year
0,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,english,US,bias,2016
1,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...,english,US,bias,2016
2,BREAKING: Weiner Cooperating With FBI On Hilla...,Red State : \nFox News Sunday reported this mo...,english,US,bias,2016
3,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...,english,US,bias,2016
4,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,english,US,bias,2016


Text preprocessing
* Similar to preprocessing for BERT training data
* additional preprocessing:
  * remove newline characters

In [ ]:
# random examples
n_doc = df_2016.shape[0]
random_idx = random.sample(range(n_doc),5)
for i in random_idx:
  print("title: " , df_2016.at[i, "title"])
  print("text: ", df_2016.at[i,"text"], "\n")

title:  “Bernie needs to be ground to a pulp.” Leaked Email to John Podesta in February
text:  Koch Brothers Secretly Allied w. George Soros for Hillary Clinton → Jay Kurtz 
Why would Bernie supporters vote Clinton? Because if she wins, his movement will be finished, and she will see to it that it is. 
On the other hand, if Trump wins, Hillary is finished, and Bernie will become the driving force in the Democratic Party, and will crush Trump in 2020. 
I just don’t understand why ANY Bernie supporter would be DUMB ENOUGH to vote for Hillary. gmatch 
There are many reasons not to vote for Hillary. Bernie is a proven fraud and should go away. If Trump wins – the DNC will have to change or it will not survive. kimyo 
while we’re talking about podesta emails, what is your interpretation of the following email from podesta re: a 2015 cnbc interview in which sanders stated “When you hustle money like that, you don’t sit in restaurants like this” and “That type of wealth has the potential to i

In [14]:
def cleanText(text):

  # remove newline characters
  text = re.sub('\n', ' ', text)

  # remove twitter handles
  text = re.sub('@[^\s]+','',text)

  # remove hashtags
  text = re.sub('#[^\s]+','',text)

  # replace multiple spaces with single space
  text = re.sub(" +", " ", text)

  # convert U.S. to US
  text = re.sub("U.S.", "US", text)
  return text

In [15]:
# remove last sentence 
def remove_last(text):
  text = re.sub('[.?!][^?|.|!]*$','',text)
  text = re.sub('[.?!][^?|.|!]*$','',text)
  text = text +"."
  return text

In [16]:
df_2016["text_cleaned"] = df_2016["text"].apply(remove_last)
df_2016["text_cleaned"] = df_2016["text_cleaned"].apply(cleanText)

df_2016["title_cleaned"] = df_2016["title"].apply(cleanText)
# add period at end of title. 
df_2016["title_cleaned"] = df_2016["title_cleaned"].apply(lambda x: x +". ")

df_2016["title_text_cleaned"] = df_2016["title_cleaned"] + df_2016["text_cleaned"]

In [17]:
df_2016 = df_2016[["title_text_cleaned", "type"]]

In [18]:
df_2016.to_csv(path+"Data/fakenews_2016/fake_cleaned.csv")

# Load Trained BERT model

In [19]:
 # load data 
 df_2016 = pd.read_csv(path + "Data/fakenews_2016/fake_cleaned.csv", index_col = 0)
 text = df_2016["title_text_cleaned"].tolist()

In [20]:
# import BERT-base pretrained model
bert = AutoModel.from_pretrained('bert-base-uncased')

# Load the BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [21]:
max_len = 512
# tokenize and encode sequences 
tokens = tokenizer.batch_encode_plus(
    text,
    max_length = max_len,
    padding='max_length',
    truncation=True
)

In [22]:
seq = torch.tensor(tokens["input_ids"])
mask = torch.tensor(tokens["attention_mask"])
y = torch.tensor([1] * df_2016.shape[0])


In [23]:
#TEST data loader
batch_size = 32

# wrap tensors
test_data = TensorDataset(seq, mask, y)

# dataLoader for validation set
test_dataloader = DataLoader(test_data, batch_size=batch_size)


In [24]:
class BERT_Arch(nn.Module):

    def __init__(self, bert):
      
      super(BERT_Arch, self).__init__()

      self.bert = bert 
      
      # dropout layer
      self.dropout = nn.Dropout(0.1)
      
      # relu activation function
      self.relu =  nn.ReLU()

      # dense layer 1
      self.fc1 = nn.Linear(768,512)
      
      # dense layer 2 (Output layer)
      self.fc2 = nn.Linear(512,2)

      #softmax activation function
      self.softmax = nn.LogSoftmax(dim=1)

    #define the forward pass
    def forward(self, sent_id, mask):

      #pass the inputs to the model  
      _, cls_hs = self.bert(sent_id, attention_mask=mask)
      
      x = self.fc1(cls_hs)

      x = self.relu(x)

      x = self.dropout(x)

      # output layer
      x = self.fc2(x)
      
      # apply softmax activation
      x = self.softmax(x)

      return x

In [25]:
# pass the pre-trained BERT to our define architecture
model = BERT_Arch(bert)

#load weights of best model
model_state_file = 'saved_weights-2.pt'
model.load_state_dict(torch.load(path + 'Data/political_news/' + model_state_file))

# push the model to GPU
model = model.to(device)

In [26]:
def prediction():
  
  print("\nGetting Prediction on Test Data...")
  
  # empty list to save the model predictions
  total_preds = []

  # iterate over batches
  for step,batch in enumerate(test_dataloader):
    
    # Progress update every 50 batches.
    if step % 50 == 0 and not step == 0:
      # Report progress.
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(test_dataloader)))

    # push the batch to gpu
    batch = [t.to(device) for t in batch]

    sent_id, mask, labels = batch



    # deactivate autograd
    with torch.no_grad():
      
      # model predictions
      preds = model(sent_id, mask)

      preds = preds.detach().cpu().numpy()

      total_preds.append(preds)

  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  return total_preds

In [27]:
preds = prediction()

# model's performance
preds = np.argmax(preds, axis = 1)
print(classification_report(y, preds))


Getting Prediction on Test Data...
  Batch    50  of    297.
  Batch   100  of    297.
  Batch   150  of    297.
  Batch   200  of    297.
  Batch   250  of    297.
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.82      0.90      9482

    accuracy                           0.82      9482
   macro avg       0.50      0.41      0.45      9482
weighted avg       1.00      0.82      0.90      9482



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Note that 2016 dataset only contains fake news articles, so we can ignore the precision and f1-scores
# We reach an accuracy of 81% 

In [28]:
# take a look at some of the mis-classified cases
incorrect = np.argwhere(preds == 0).tolist()
incorrect = [item[0] for item in incorrect]

In [29]:
df_incorrect = df_2016.iloc[incorrect,:]

In [30]:
df_incorrect.to_csv(path + 'Data/fakenews_2016/misclassified.csv')

In [31]:
## some examples of incorrect classifications (fake news classified as real news)
random_idx = random.sample(incorrect, 5)
for i in random_idx:
  print(df_2016.at[i,"title_text_cleaned"])

The Chronicles of Carlos Danger. As the FBI assesses emails from former congressman’s devices, his sex scandal is once again making headlines. Joe Biden’s response: ‘Oh, God’ By Oliver Milman The Guardian October 31, 2016 As a fractious and often sordid presidential campaign reaches its denouement, Hillary Clinton may be reflecting upon the grim irony that it is Anthony Weiner who has provided what could be the final, miserable twist. Weiner’s predilection for sending sexually laden text messages derailed his career in Congress and his chances of becoming mayor of New York City , but his soap-opera-style re-emergence in the 11th hour of the presidential race may yet trigger the greatest political carnage of all. His appearance comes in an election already defined by allegations of inappropriate sexual behaviour by Clinton’s husband, former president Bill Clinton, and by the Republican nominee, Donald Trump . On Friday, Clinton called on the FBI to “immediately” explain its announcement

In [32]:
df_2016.iloc[random_idx,:]

,title_text_cleaned,type
4301,The Chronicles of Carlos Danger. As the FBI as...,bs
3743,UK Economy Grows 0.5% in Three Months after Br...,conspiracy
8852,Hillary Clinton’s Wall Street Fundraising Bene...,bs
1934,Powerful earthquake hits central Italy for the...,bs
1255,How the West Provoked the New Cold War. How th...,bs


In [ ]:
df_2016.iloc[random_idx,:]

,title_text_cleaned,type
2991,"The Email Mess, Comey’s quandary and the expan...",bs
7189,These New Solar Panels Use Sunlight to Create ...,bs
649,Jordan Troops Kill Three US Military Trainers ...,bs
2588,From Bad to Worse: Obama’s Ransom Payment to I...,hate
2204,Comment on WikiLeaks emails show Clinton aide ...,bs


In [ ]:
# This was incorrectly classified as real article.
# original article in (http://ghovexx.blogspot.com/2016/10/snapchat-to-raise-up-to-4-billion-in.html)
# however, note that this is based on a true article (https://www.bbc.com/news/world-middle-east-37871836)
df_2016.at[649, "title_text_cleaned"]

"Jordan Troops Kill Three US Military Trainers at Air Base. Jordan Claims US Vehicle 'Failed to Stop' at Gate by Jason Ditz, November 04, 2016 Share This Three US soldiers assigned to a training operation in Jordan were killed today by Jordanian troops at the al-Jafr Air Base, when base security opened fire on them near the gate. One Jordanian soldier was also reported wounded in the incident. Unlike other recent incidents of US soldiers killed by allies, this incident was not chalked up to infiltration by rogue elements. Rather, Jordanian officials blamed the US soldiers, claiming their vehicle failed to stop at the gate to the air base and security reacted. The Pentagon was a little less cut and dry on the matter, saying they can’t say for sure if it was a “deliberate” attempt to kill US soldiers or some sort of misunderstanding. They say they are working with the Jordanian government to gather more details. US trainers in Jordan are there to train Syrian rebels, an operation which h

In [ ]:
#
df_2016.at[2588, "title_text_cleaned"]

'From Bad to Worse: Obama’s Ransom Payment to Iran is Just the Tip of the Iceberg. Search form Search From Bad to Worse : Obama’s Ransom Payment to Iran is Just the Tip of the Iceberg In his bid to pursue a legacy, Obama charts disastrous course with reckless abandon. November 2, 2016 Ari Lieberman Most of us, including several democratic lawmakers, cringed when Obama inked the Iran deal but those of us who are familiar with the malevolent nature of the Iranian regime, recoiled in horror when we learned that Obama transferred $400 million to the Iranians in exchange for four American citizens held captive by the mullahs on trumped up charges. The $400 million was part of a larger installment totaling $1.7 billion, ostensibly to settle claims Iran had against the United States stemming from aborted Iranian arms purchases dating back to the Shah. Obama claimed that this was money that was “owed” to Iran and the settlement, which included $1.3 billion in interest, saved the US taxpayer “b

In [ ]:
# failed to identify fake science. From undergroundreporter2016.
# actually, this seems to be real too.. 
df_2016.at[7189,"title_text_cleaned"]

'These New Solar Panels Use Sunlight to Create Clean Drinking Water from the Air. By Christina Sarich Jordan — You’ve already heard of solar panels that can sustainably provide energy, but what about an ultra-high tech solar panel that can provide doubly-pure, twice distilled water for a family of four, out of thin air? A new start up company called Zero Mass Water that won’t rely on outdated municipal water with decaying pipes full of lead to deliver water to the ‘middle billion’ and under-served people lacking clean drinking water in the world. It will create it with solar panels and a technology that capitalizes on moisture in the air. The company’s tag line is ‘drinking water democratized,’ and it certainly seems to stand for the exact opposite world view of, say, Nestle , which has been stealing water from people and San Bernardino National Forest reserves and then selling it back to people in plastic water bottles. Zero Mass Water can deliver clean water to people in the poorest 

In [ ]:
df_2016.at[2204,"title_text_cleaned"]

'Comment on WikiLeaks emails show Clinton aide says Foundation paid for Chelsea’s wedding by Dr. Eowyn. Posted on November 7, 2016 by DCG | 3 Comments That’s how the Clintons roll… From Fox News : Former President Bill Clinton’s top aide wrote in 2012 that Chelsea Clinton used Clinton Foundation resources “for her wedding and life for a decade” and a top Foundation donor was responsible for “killing” unfavorable press coverage – all as an internal Foundation audit uncovered numerous conflicts of interest and “quid pro quo benefits,” according to emails released Sunday by WikiLeaks. Doug Band, founder of global strategies company Teneo and Bill Clinton’s personal assistant since the 1990s, wrote the Jan. 4, 2012, email to future Hillary Clinton presidential campaign chair John Podesta and two other Clinton aides after receiving word that Chelsea had told “one of the [President] bush 43 kids” and others about “an internal investigation of money within the foundation.” Band wrote such cha

In [ ]:
# this example is more of a "bias/ hatred"
df_incorrect.at[23,"title_text_cleaned"]

'LOL! BRITISH WIFE Of LIB ACTOR Who Said: “There Will Never Be A President Donald Trump”…Warns Americans About President-Elect Trump [VIDEO]. Go to Article Political activist and Hillary supporter Amal Clooney traveled to Texas to warn Americans about our elected President Donald J. Trump and his tough immigration policies. Maybe she didn’t get the memo…Trump’s tough immigration stance is what catapulted him to the top of the heap in the crowded pack of GOP primary contenders. It wasn’t too long ago that her arrogant liberal husband announced with great confidence, “There will never be a President Trump”. Perhaps Amal should’ve traveled to some of the border towns if she wanted to get a clear picture of how illegal immigration is affecting our nation. Aliens crossing our borders illegally have destroyed the property of American ranchers, and threatened the safety and security of American citizens living in homes in or near the US -Mexican border towns… Amal Clooney has warned that many

In [ ]:
# This was incorrectly labled as real news
# but also seems like its based on a true story? 
#https://www.bloomberg.com/news/articles/2016-10-26/snapchat-said-to-seek-to-raise-as-much-as-4-billion-in-ipo
df_2016.at[9475,"title_text_cleaned"]

"Snapchat To Raise Up To $4 Billion In IPO, Valuing Company As Much As $40 Billion. Oct 26, 2016 3:52 PM 0 SHARES The long await IPO of Snapchat is finally coming: according to Bloomberg the social media will seek to raise as much as $4 billion in its planned initial public offering, making it the biggest social media company to go public since Twitter's initial public offering in November 2013. Bloomberg reports that The IPO could value Snapchat at about $25 billion to $35 billion, citing unnamed sources, and while no final decision has been made and the size of the IPO may change, and the valuation could reach as much as $40 billion, one of the people said. In a surprising twist, because the company’s revenue is less than $1 billion, it plans to file IPO documents confidentially with the US Securities and Exchange Commission, one of the people said. Snapchat chose Morgan Stanley and Goldman Sachs Group Inc. to lead its offering, people familiar with the matter said earlier this month

Overall, I'm not sure if we can trust the 2016 fake news data source. It was collected from this chrome extension called "BS detector". It's a collection of articles classified as "BS" by this detector, so it could contain some false positives (real articles mistakenly labeled as "BS")

#2. External validation from other fake news sources

* From Onion and Real News Right Now



In [ ]:
new_new = ['COVID-19: AFGHAN GOVERNMENT FORCES WOMEN TO WEAR THREE LAYERS OF BURQAS TO STOP THE CONTAGION. Facing a shortage of disposable masks and a surge in Covid-19 cases, the Afghan Ministry of Public Health has decreed today that women across the country have to wear three burqas, one on top of the other, when outside their residence. The Afghan Health Minister, Dr. Ferozuddin Feroz, announced a new set of restrictive measures for the population to curb the spread of the coronavirus in the country. Among these measures, the government has imposed on Afghan women the obligation to wear a minimum of three layers of burqas when in public, announcing fines of up to 20,000 Afghanis (262 US dollars) and 50 lashes for any violations. Dr. Feroz says this measure is meant to limit the propagation of the disease and protect women. “We know that women are by definition a vulnerable population so we are trying to protect them from harm.” Dr. Feroz explained that one burqa is clearly insufficient to protect its wearer, so that is why he decided to impose a minimum of three. “I understand that it might be harder to breathe or see under so much fabric, but if you can’t breathe, you can’t catch an airborne virus. It’s simple science.” Disposable masks have been distributed with scarcity in Afghanistan, as the country is already facing shortages of several types of medical equipment. Several Women’s Rights organizations have denounced the measure as misogynistic. In a brief press statement, the International Council of Women (ICW) criticized the Afghan government for “putting the responsibility of stopping the pandemic on Afghan women’s shoulders”. The Executive Director of the United Nations Entity for Gender Equality and the Empowerment of Women (often called UN Women), Phumzile Mlambo-Ngcuka, also denounced the measure. In a brief communiqué, the head of UN Women said her organization would remain extremely vigilant as “some governments are using the pandemic to push their own political agendas through their restrictive measures.”',
           'Insider Report Reveals Obama Tried Persuading Ginsburg To Die During His Presidency. WASHINGTON—Shedding new light on the former president’s relationship with the Supreme Court justice, an insider report released Tuesday revealed that Barack Obama repeatedly tried persuading Ruth Bader Ginsburg to die during his presidency. “The former president was frank with Justice Ginsburg about the potential risks of her continuing to remain on the court, which is why he urged her to drop dead while he was still in office,” read the report in part, detailing Obama’s arguments that Ginsburg could ensure her legacy was maintained by gracefully getting sucked into a whirlpool or devoured by wolves early in his second term. “Obama wanted to make the transition as easy as possible for Ginsburg by suggesting a wide variety of options for how she might want to go out, such as the numerous poisons he had brought along to their meeting in addition to a loaded handgun and a length of rope.” The report concluded that Ginsburg ultimately demurred, opting to remain on the Supreme Court after tripping on Obama’s foot and surviving a fall down 30 flights of stairs.',
           'Trump Family To Halt Big Bath They All Take Together Every Night During President’s Quarantine. WASHINGTON—Citing an abundance of caution following President Donald Trump’s coronavirus diagnosis, the White House announced Wednesday that the Trump family would be discontinuing its practice of taking a big bath together every night until the president recovered. “We are exercising every precaution to stop the spread of Covid, so until his quarantine is over, the president and his family will halt their beloved tradition of all piling into a giant tub for a nighttime washing,” said White House chief of staff Mark Meadows, confirming that the Trumps would be unable to slosh around in the warm water and line up to get their backs scrubbed by a soapy loofah for at least the next 10 days. “We have implemented new safety procedures such as moving the basin to an open-air location on the front lawn and sterilizing all rubber duckies and toy submarines between baths in order to ensure family tub time can resume as soon as possible.” At press time, Don Jr. and Eric had reportedly taken advantage of the two weeks without a bath by getting absolutely filthy playing in the White House Rose Garden.',
           'Donald Trump Reprimanded For Continually Interrupting Harris, Moderator During VP Debate. SALT LAKE CITY—After he issued a relentless stream of combative remarks that were criticized as unhelpful and distracting, President Donald Trump was reprimanded Wednesday night for continually interrupting Sen. Kamala Harris and moderator Susan Page during the vice presidential debate. “Excuse me, Mr. President, but please remember your campaign agreed to the format of this debate between Sen. Harris and Vice President Pence, and your repeated interruptions are a clear violation of those rules,” Page said to Trump, who accused the moderator of trying to silence him when she asked Harris about her position on Medicare for All, and then complained that the rules were only being enforced against him. “Sir—sir, in all fairness, you’ve been interrupting quite a bit more than anyone else on stage. In fact, you’ve had more time to speak than the two vice-presidential candidates combined. The country would be better served if you could just set down the microphone now.” At press time, Trump had turned his attention to ridiculing Mike Pence, whose performance he derided as low energy, weak, and “terrible television.”',
           'Pence Instinctively Addresses Harris’ Husband In Audience During Debate Responses. SALT LAKE CITY—As he made direct eye contact with the man sitting in the audience nearly 30 feet away, Vice President Mike Pence appeared to instinctively address his responses to Sen. Kamala Harris’ husband during Wednesday night’s vice-presidential debate. “Excuse me, sir, but I take objection to your wife’s characterization of this administration’s priorities,” Pence said as he turned away from his opponent and pointed a finger at her spouse, entertainment lawyer Doug Emhoff, claiming that the policies he allowed his wife to support would be disastrous for the country. “Could you please explain to her that my experience as chair of the president’s Coronavirus Task Force gives me a far greater grasp of healthcare issues? To be honest, I’m not sure why you are letting her talk to me like this, in such contrary tones. It might be easier if you could just address me directly.” Pence later chastised Harris for interrupting her husband during his response.',
           'Nation’s Bland, Ineffectual White Supremacists Waiting For Signal From Pence. WASHINGTON—Ready to spring into action at a moment’s notice, the nation’s bland, ineffectual white supremacists confirmed Friday they were just waiting for the signal from Vice President Mike Pence. “Just give us the word, and we’ll shake our heads while murmuring, ‘This country isn’t what it used to be,’” said 41-year-old father of three Thomas Strom, who spoke on behalf of the country’s millions of milquetoast white nationalists, warning they were mobilized at their dinner tables, living room couches, and desktop computer keyboards. “Vice President Pence, we’re fully prepared to complain about virtue signaling on our Facebook accounts and harrumph behind strangers in line at the grocery store. We’re poised to roll our eyes when someone brings up Spanish class at the next PTA meeting and stare at the minorities at the gas station. You can count on us, sir. We have your back.” At press time, the nation’s bland white supremacists had frantically scrambled in all directions after a Black man looked at them.',
          'Trump Prepares For Next Debate With Help Of Dexamethasone-Induced Hallucinations. WASHINGTON—In an effort to be as rehearsed as possible for the rescheduled rematch against Joe Biden, President Donald Trump was reportedly hard at work preparing for his next debate Thursday with the help of a team of Dexamethasone-induced hallucinations. “Biden is definitely going to hammer you for being so big and strong and smart, so be prepared to seize the spotlight,” said the floating head of Herman Caine, joining a phantasmagorian Kermit The Frog, Richard Nixon, and dozens of other faces flashing through the president’s brain to urge Trump to get on the offensive as soon as the debate stage stopped spinning and materialized from the darkness. “Remember, the vice president is going to try and distract you by bursting open into a thousand bats or becoming a giant and swallowing you whole, so be sure to have a pithy response on how to keep our cities safe. And whatever you do, try not to let the flesh melt off your body until you’re only a skeleton like you did during the last practice session, as this could be looked upon as a sign of weakness.” At press time, the debate prep had been halted after Trump became furious over conflicting advice from a snake-covered George Washington and himself as a two-year-old.',
           'Catholic Church to Alter Image of Jesus Over Allegations of White Privilege. WASHINGTON, D.C. – In an effort to provide a more inclusive experience for worshipers, Catholic churches throughout the United States will undergo a drastic transformation this summer as they move to offer a more realistic and culturally appropriate version of Jesus than the historically European depiction embraced by the Church. The decision by the United States Conference of Catholic Bishops to portray Jesus as a brown-skinned Middle Eastern man clad in sandals and an off-white tunic comes on the heels of nationwide movement led by the American Leadership Council of Progressive Christian Womyn to eradicate perceptions of white privilege from the Christian faith. “For far too long we have been taught that whiteness is a prerequisite for holiness,” the group’s co-founder, Kathryn Lockhart, told Real News Right Now. While adamant that she was in no way discrediting the teachings and philosophy of Jesus of Nazareth, Ms. Lockhart argued that Christ “only became the Son of God by inheriting daddy’s power. Robert Langdon, a professor of Art History and Symbology at Harvard University called the decision by the U.S. Conference of Catholic Bishops “a little silly,” but acknowledged that white privilege likely played a prominent role in the establishment of Christianity as a global religion. “When you have a situation where a white man living in the Middle East in 27 A.D. spontaneously launches a successful two-thousand-year-old religion, it raises a lot of questions.” With thousands of souls put to death under Pontius Pilate, experts say it stands to reason that Jesus Christ – the only Caucasian man in all of Palestine who was not a Roman citizen – received special treatment and favors which, unlike countless other condemned Palestinian men and women, allowed him to miraculously rise from the dead and helped facilitate his speedy ascension into heaven.',
          'Experts Say Millennials Are Inhaling Too Much Oxygen. CAMBRIDGE, Ma. – While Millennials account for just twenty-two percent of the U.S. population, researchers at the Massachusetts Institute of Technology say that in the age of COVID-19, Millennials are consuming far more oxygen than the earth is capable of producing. “Current trends show Millennials are on track to inhale upwards of twenty-five tons of oxygen per year -more than twice the global average,” explained Dr. Ross E. Geller, the study’s chief researcher. Geller, who holds a PhD in paleontology from Columbia University, warned that oxygen consumption in the United States alone has risen by more than 700% since February. Due to social distancing guidelines and stay-at-home orders, Millennials -often referred to as the Privileged Generation- are leaving their apartments in droves, gallivanting through otherwise deserted streets and neighborhoods, and assembling in parks and other public spaces for the purpose of exercising, socializing, and other “extracurricular” activities such as vaping and dog walking. “Our planet only produces so much oxygen per year,” said Dr. Gordon Freeman, an MIT alumni and researcher who holds a PhD in theoretical physics. “Millennials nowadays have so much free time on their hands due to COVID-“19, and, as we’re seeing, the vast majority of the ‘elitist generation’ insist on being outdoors where they’re sucking up sixty to seventy percent of the world’s oxygen supply.” As scientists race to find a solution to the Millennial question, experts say older generations may soon find themselves short on oxygen. “Those in the greater and greatest generations could be inhaling close to 100% carbon dioxide by the end of the year,” Dr. Geller predicted. However, younger generations such as Z and Alpha are known to have much smaller lungs than full grown adults and therefore require very little, if any, oxygen to survive.'
          'France, Germany Strike Deal with DNC to Influence 2020 Election in Favor of Democratic Nominee. WASHINGTON, D.C. – Following a historic vote by the Senate last week to sanction the solicitation of foreign interference in U.S. elections, the Democratic National Committee on Wednesday announced it has struck a deal with France and Germany to influence the 2020 presidential election in favor of the Democratic nominee. “Congress has made clear that if a political candidate or party strongly believes that soliciting foreign interference in an election is in the best interest of the country, then doing so is both constitutionally and legally acceptable,” DNC chairman Tom Perez announced during a press briefing at the party’s headquarters in Washington, later adding, “We finally have a level playing field.” Under the terms of the agreement, French and German intelligence agencies will launch a joint operation designed to influence the U.S. presidential election in favor of the Democratic presidential nominee. In exchange, the DNC has provided a guarantee that the incoming administration will significantly increase funding and support for NATO. “Think of it as a kind of quid pro quo”, explained Mr. Perez who later added that France and Germany will have the unique opportunity to “heavily” influence U.S. foreign policy in Europe and the Balkans. Speaking to CNN’s Jake Tapper on Tuesday, South Korean president Moon Jae-in confirmed that his country has also agreed to help influence the 2020 election. “We view it as a long-term investment in the safety and security of the Korean peninsula,” President Moon told Tapper, adding that South Korea has agreed to hack U.S. voting machines and convert “red votes to blue” in exchange for a forty percent discount on future arms sales and an increased U.S. military presence along the DMZ. Despite ongoing tensions between Tehran and Washington, a senior DNC official who spoke to Real News Right Now on condition of anonymity said Democratic operatives have been engaged in direct talks with the Iranian regime in hopes of securing additional support in the months leading up to the 2020 presidential election. “We’re still ironing out the details, but the gist of it is Iran has agreed to launch a disinformation campaign against the Trump White House in exchange for the lifting of all sanctions,” the official told Real News Right Now, continuing, “Given the significance of this election, all options are on the table.”',
           'Barr Appoints Special Counsel to Determine Validity of Obama’s Birth Certificate. WASHINGTON, D.C. – U.S. Attorney General William Barr has announced the appointment of a special prosecutor to oversee a Justice Department investigation into the authenticity of former president Barack Obama’s birth certificate. “This is something that President Trump and I both agree is long overdue,” Attorney General William Barr told reporters during a press briefing at the Department of Justice on Tuesday. “The president has asked me to appoint a competent nonpartisan prosecutor to spearhead the investigation,” Attorney General Barr explained. Mr. Barr went on to say that due to “the desolate state” of the DOJ, he saw no other option but to appoint himself as special counsel for the birth certificate investigation. “I cannot, in good faith, trust anyone other than myself to lead an investigation of this magnitude,” the attorney general concluded. While Mr. Barr declined to elaborate on the nature of the investigation, he said Justice Department lawyers are preparing subpoenas for a number of individuals believed to be associated with the birth of Barack Hussein Obama on August 4, 1961. “In the coming weeks we will issue subpoenas for those individuals charged with record keeping at the Honolulu hospital where Mr. Obama was allegedly born,” the attorney general stated, adding that the doctors who “allegedly delivered” the infant Obama -or their next of kin- will also be called to testify. In an official statement released Wednesday, former CIA director and acting Secretary of State Mike Pompeo said the Department of State is lending “limited support” to Mr. Barr’s investigation. “We at the State Department are working closely with our counterparts in the Kenyan government to locate a witch doctor who is suspected of masterminding the plot to conceal Barack Obama’s national origin,” the statement read. “This is about seeking absolute confirmation that Barack Obama was actually eligible to hold office from 2008 until 2016,” a senior White House official, who spoke on condition of anonymity because they were not authorized to speak to the press, told The Washington Times. “In the end we’re looking at the immediate removal of all Obama-era appointees including federal judges and Supreme Court Justices; not to mention a full reversal of any Obama policies and pardons that President Trump may have missed or overlooked,” the official told The Times. Although both the Obama White House and the state of Hawaii have produced copies of Mr. Obama’s long-form birth certificate, President Donald J. Trump believes both to be forgeries and has long called for an investigation into their legitimacy. “They’ve got an entire black market for fake birth certificates,” then-candidate Trump told Fox News host Chris Wallace in 2015, adding, “It’s a terrible thing what they’re doing on that island.” The Department of Justice investigation into the validity of Barack Obama’s birth certificate -along with his status as an American citizen- is expected to last at least two and a half years and will cost U.S. taxpayers upwards of $250 million.'

]

In [ ]:
new_new = [cleanText(item) for item in new_new]

In [ ]:
max_len = 512

# tokenize and encode sequences in the training set
tokens_new = tokenizer.batch_encode_plus(
    new_new,
    max_length = max_len,
    padding='max_length',
    truncation=True
)

# convert lists to tensors
new_seq = torch.tensor(tokens_new['input_ids'])
new_mask = torch.tensor(tokens_new['attention_mask'])

with torch.no_grad():
  new_preds = model(new_seq.to(device), new_mask.to(device)) #push to gpu
  new_preds = new_preds.detach().cpu().numpy()

In [ ]:
np.argmax(new_preds, axis = 1)

array([0, 0, 0, 0, 0, 1, 1, 1, 1, 0])

# Most viewed faked news stories shared on facebook 2019

*https://www.businessinsider.com/most-viewed-fake-news-stories-shared-on-facebook-2019-2019-11



In [35]:
d = {'title': ["Trump's grandfather was a pimp and tax evader; his father a member of the KKK",
               "Pelosi Diverts $2.4 Billion From Social Security To Cover Impeachment Costs",
               "Ocasio-Cortez Proposes Nationwide Motorcycle Ban",
               "Trump Is Now Trying To Get Mike Pence Impeached",
               "Omar Holding Secret Fundraisers With Islamic Groups Tied to Terror",
               "BREAKING: Nancy Pelosi’s Son Was Exec At Gas Company That Did Business In Ukraine",
               "Dems Vote To Enhance Med Care for Illegals Now, Vote Down Vets Waiting 10 Years for Same Service",
               "Joe Biden Calls Trump Supporters the “Dregs of Society”"],
     'text':['Most families of enormous wealth have a dark and sometimes scandalous, even monstrous past. Donald Trump\u2019s clan is no exception to that rule. His grandfather was a pimp and a tax evader, his father a racist who would in the course of his life, clash with New York City Police as a member of the Ku Klux Klan and then as a wealthy real estate magnate, refuse to rent to people of color. Donald Trump\u2019s legacy is anything but a rag to riches story. His dad kicked the bucket with $250-$300 million in the bank. The man who wants to ban all people of a particular religion from travel wasn\u2019t born with a silver spoon in his mouth, his was white gold. The only thing more obnoxious than Donald Trump himself, is his family\u2019s money-grubbing, bigoted history. Donald Trump\u2019s grandfather, Frederick or Friedrich Trump, made his money from operating a decadent restaurant and hotel during the Gold Rush at Klondike in the Yukon. That\u2019s a nice way of saying it. "Trump made his first fortune operating boom-town hotels, restaurants and brothels", is more accurate, according to the CBC news report, "Donald Trump\u2019s grandfather ran Canadian brothel during gold rush, author says". Author Gwenda Blair simply wrote, "The bulk of the cash flow came from the sale of liquor and sex." Trump\u2019s grandfather was born in Germany, to parents who were employed by a vineyard. He moved to New York City in 1885 where he became a barber. After six years of this, Frederick Trump moved across the United States to Seattle, Washington, where he owned and operated what he referred to as a "decadent restaurant" that was actually called "Poodle Dog" in Seattle\u2019s red light district. Interestingly, the name and concept that had already been established in San Francisco. (He named his restaurant after a dog but would later make money selling horse meat) Around this time Frederick Trump became a US citizen. A Yukon Sun Newspaper writer described his business: "For single men the Arctic has excellent accommodations as well as the best restaurant in Bennett, but I would not advise respectable women to go there to sleep as they are liable to hear that which would be repugnant to their feelings – and uttered, too, by the depraved of their own sex". Trump moved to Monte Cristo, Wash. in 1894, and then four years later, shortly after the Klondike gold rush began, he relocated again to Bennett, British Columbia.  Here he ran the "Arctic Restaurant and Hotel".  He would next build the "White Horse restaurant and Inn" in Whitehorse, Yukon. An article published this year by Politico, explains that Frederick Trump sold off his investments and returned to Germany in 1901, as he sensed the end of the gold rush and a subsequent end to prostitution.  The following year, he married his former neighbor, Elizabeth Christ in his native German town of Kallstadt.  Then he came under heavy scrutiny by the German government, The country had compulsory military service for men which had to be fulfilled by the age of 35.  Donald Trump\u2019s grandfather waited until he was 35 to go back to Germany.  He had already amassed great wealth worth well more than half a million US dollars, or 80,000 marks.  While his town council was eager to keep Trump and his money, who billed himself as a man who "avoided bars" and led "a quiet life", other German authorities had a different plan, the Politico article explains.  In their view, Trump had relocated to Germany in order to avoid both tax and military-service obligations. The fact that Donald Trump has done so well in the Republican polls is quite amazing, as the party that we think of as conservative, would be expected to shun a man who created exploitative beauty pageants and was born rich strictly because of the nefarious activities of his ancestors. Donald Trump has often said he made his money "the old-fashioned way," and this is true, in that he prospered from racism. New York Times article published 01 June 1927, related Donald Trump\u2019s father Fred Trump\u2019s role in a Ku Klux Klan brawl that pitted 1,000 klansmen against 100 New York City Police in Queens.  Though he wasn\u2019t officially charged, Fred Trump was one of seven klansmen arrested during the incident.  It probably wasn\u2019t very shocking at the time as America\u2019s racist practices were in full swing generations after Abe Lincoln freed the country\u2019s African-American slaves.  In fact, the mid-20\u2019s saw a peak in KKK activity.  Donald Trump would later deny his father\u2019s involvement in the KKK brawl in spite of the fact that it happened two decades before he was born.  Fred Trump\u2019s enthusiasm for racist practices never changed until he was forced to do so by the law. Presidential Candidate Donald Trump joined his father\u2019s real estate company in New York in 1971, and only two years later, the company was served with a civil rights lawsuit that was filed against the Trump organization because it refused to rent to Black people.  The Urban League got wind of the racist rental policy and actually sent both Black and White people in to apply for apartments that belonged to complexes owned by the Trumps.  What they proved, was that Black people were denied rentals across the board, and only Whites were approved.  A Village Voice article by Wayne Barrett, published in 1979, blew the lid off the Trump organization\u2019s brewing pot of racist practices. The article explains that Trump\u2019s reaction was to claim that the suit was a "nationwide drive to force owners of moderate and luxury apartments to rent to welfare recipients." "\u2019We are not going to be forced by anyone to put people...in our buildings to the detriment of tenants who have, for many years, lived in these buildings, raised families in them, and who plan to continue to live there. That would be reverse discrimination,\u2019 he said. \u2019The government is not going to experiment with our buildings to the detriment of ourselves and the thousands who live in them now.\u2019"Indeed, Trump\u2019s wild, largely uninformed and unintelligent rants are the legacy of men who walked over the backs of other Americans to gain and secure their wealth.  This may be symbolic of the American capitalist way, but it falls short of any form of greatness or real human success.  As they say, the apple doesn\u2019t fall far from the tree.',
             'Freedom isn\u2019t free, the saying goes.  And neither is the opposite of freedom, which for today\u2019s purpose, is the impeachment of President Donald Trump. Counting legal fees for a slew of legal teams, experts, time, maintenance, and constant deliveries of Chardonnay, you and I, Mr. and Mrs. Taxpayer, are looking at about 2 and a half billion dollars. Hope your boss can give you a raise. Speaker of the House Pelosi isn\u2019t too keen on having that number creep out into the public eye, so yesterday, as evidenced by the filing of re-appropriation form 77-A in the Congressional Budget office, she quickly moved the amount from the Department of Social Security to cover the cost. Are you retired?  Well get ready to pay, bitch. Independent journalist Jay Garrick sped to Pelosi\u2019s office and got a brief statement: “Well most of Trump\u2019s voters and supporters are retired seniors.  I think it\u2019s fitting that we ask them to help pay to fix the mistake they made.  They\u2019ll just have to do without cable for a year or two.  Okay?  Now excuse me, I have to meet up with my liquor boy.” The price tag for this little dog and pony show seems awfully steep, especially with only a little more than half of Americans approving it.  Let\u2019s hope that at the very least, we get a T-shirt.',
             'Junior Congresswoman Alexandra Ocasio-Cortez has made a lot of waves recently with her Green New Deal nonsense and child-like proposals and speeches.  Well, it\u2019s no different this afternoon as the political pixie announced her intention to draft legislation banning motorcycles from use in the United States of America. The Senorita of Socialism threw out all manner of statistics regarding deadly accidents and injuries, relaxed traffic rules and tolls for bikers, as well as a not-so-veiled jab at a core demographic of President Trump\u2019s supporters : “Besides like, what I just said?  A lot of these like, motorcycle people, okay, they\u2019re like : ‘Ooh, look at me, I\u2019m all old and fat and tough and I voted for Trump and smell like wet dog.\u2019  And I\u2019m supposed to slow my Prius down so you can like, noise pollute past everyone?  I mean some of us have a nail appointment, people.” Opposing the ban, spokesman and leader of “Bikers For Trump”, Clee Torres, also gave a statement to the press, expressing his, and his organization\u2019s views : “I don\u2019t know where they found this little girl with the cute little mouth, but ain\u2019t nobody taking away our hogs.  President Trump is the best damn thing to happen to this country since Zima.” Torres added that Bikers For Trump would hold a protest against the proposal with a million motorcycle convoy in Washington, previous incarnations of which have drawn tens of participants.  Cortez may want to get herself back behind the bar where her little Punky Brewster act can make her some tips.',
             'During a press conference, Trump said that if he is going to be investigated, Vice President Pence\u2019s phone calls with Ukraine should also be investigated. According to the official White House transcript as provided to PolitcusUSA, Trump said: the witch hunt continues, but they\u2019re getting hit hard in this witch hunt, because when they look at the information, it\u2019s a joke. Impeachment? For that? When you have a wonderful meeting, or you have a wonderful phone conversation? I think you should ask. We actually — you know, that was the second conversation. I think you should ask for the first conversation also. I can\u2019t believe they haven\u2019t, although I heard there\u2019s a — there\u2019s a rumor out they want the first conversation. It was beautiful. It was just a perfect conversation. But I think you should do that. I think you should do, and I think you should ask for VP Pence\u2019s conversation because he had a couple conversations also. I can save you a lot of time. They\u2019re all perfect. Nothing was mentioned of any import other than congratulations. But the word is that they\u2019re going to ask for the first phone conversation. You can have it anytime you need it. And also Mike Pence\u2019s conversations, which were, I think, one or two of them. They were perfect. They were all perfect. There should be no doubt in anyone\u2019s mind that Trump would throw Mike Pence under the bus in a second to save his presidency. Trump has made no secret of his desire to build a family political dynasty, and he sees his kids Don Jr. and Ivanka as next line after he leaves office. Trump has declined to endorse Pence, and at times, has even waffled on keeping Pence as his vice president. Trump is looking to blame anyone, including people in his own administration to save his skin. Pence was always going to go down with the Trump ship. But, he never thought that it would be Trump pushing him overboard.',
             'Democratic freshman Rep. Ilhan Omar (D., Minn.) has been holding a series of secret fundraisers with groups that have been tied to the support of terrorism, appearances that have been closed to the press and hidden from public view. The content of these speeches, given to predominately Muslim audiences, remains unknown, prompting some of Omar\u2019s critics to express concern about the type of rhetoric she is using before these paying audiences, particularly in light of the lawmaker\u2019s repeated use of anti-Semitic tropes in public. Omar recently spoke in Florida at a private event hosted by Islamic Relief, a charity organization long said to have deep ties to groups that advocate terrorism against Israel. Over the weekend, she will appear at another private event in California that is hosted by CAIR-CA PAC, a political action committee affiliated with the Council on American Islamic Relations, or CAIR a group that was named as an unindicted co-conspirator in a massive terror-funding incident. Omar\u2019s appearance at these closed-door forums is raising eyebrows in the pro-Israel world due to her repeated and unapologetic public use of anti-Semitic stereotypes accusing Jewish people of controlling foreign policy and politics. With Omar\u2019s popularity skyrocketing on the anti-Israel left, it appears her rhetoric is translating into fundraising prowess. It remains unclear what Omar has told these audiences in her private talks. Washington Free Beacon attempts to obtain video of past events were unsuccessful, and multiple local news and television outlets in the Tampa, Fla., area, where Omar spoke to Islamic Relief last month, confirmed they were unable to gain access to the closed door event. Islamic Relief has come under congressional investigation for what lawmakers have described as its efforts to provide assistance to terrorist group such as Hamas and the Muslim Brotherhood. The charity has been banned by some countries as a result of these ties. In 2017, Congress sought to ban taxpayer funds from reaching the charity due to these terror links. A representative from Islamic Relief declined to provide the Free Beacon with any material related to Omar\u2019s appearance. "The event was closed to the media. No materials are available," the official said. On Sunday, Omar will hold another meet and greet in Irvine, Calif., for CAIR-CA PAC. Those wishing to hear Omar speak are being asked to donate anywhere from $50 to $250 dollars, according to a flyer for the event. The CAIR event also appears closed to the press. Free Beacon attempts to contact the organizer and obtain access were unsuccessful. Requests for comment on the nature of the speeches sent to Omar\u2019s congressional office also were not returned. CAIR, a Muslim advocacy group known for its anti-Israel positions, was named by the U.S. government as part of a large network of groups known to be supporting Hamas. CAIR has been cited by the Anti-Defamation League, or ADL, for using its network of supporters to promote an "anti-Israel agenda." "CAIR’s anti-Israel agenda dates back to its founding by leaders of the Islamic Association for Palestine (IAP), a Hamas affiliated anti-Semitic propaganda organization," according to the ADL. "While CAIR has denounced specific acts of terrorism in the U.S. and abroad, for many years it refused to unequivocally condemn Palestinian terror organizations and Hezbollah by name, which the U.S. and international community have condemned and isolated." Sarah Stern, founder of president of the Endowment for Middle East Truth, or EMET, a pro-Israel group that has condemned Omar for promoting anti-Semitic conspiracy theories, told the Free Beacon that the private nature of these events before controversial Islamic groups is very concerning. "I just wonder what is Rep. Omar saying to a group of Islamic supporters that she feels is so secretive that she cannot say it to the American people, as a whole?" Stern wondered. "What is so secretive that it has to be off the record and closed to reporters? Will she say the same things in public to her Jewish voters in Minnesota that she says to her Islamic friends? What does this tell us about her openness, her honesty and her integrity?" One veteran Republican political operative expressed concern about the secretive nature of these talks, telling the Free Beacon that Democrats must hide behind-closed-doors to appease these groups with anti-Israel rhetoric. "Of course she\u2019s holding these meetings in secret. That\u2019s just how Democrats roll these days," the source said. "They\u2019re for limiting your ability to spend money on the candidates you want to support, and for secretly fundraising from Islamist groups who support them. It really puts their support for campaign finance reform into perspective." After last month\u2019s Islamic Relief event, Stern\u2019s EMET and many other pro-Israel groups penned a letter to Democratic leaders in the House demanding Omar\u2019s removal from the powerful House Foreign Affairs Committee. These groups argued that Omar\u2019s anti-Semitic rhetoric and secretive meetings should disqualify her from a seat on that committee, which oversees the U.S.-Israel military alliance. "Rep. Omar\u2019s presence as a keynote speaker to raise funds for Islamic Relief USA, whose parent organization and chapters have documented ties to terrorist organizations, demonstrates that she has learned next to nothing over the last few weeks when she was reprimanded by your office and by other Democrats for posting ugly, anti-Semitic attacks on Jews and their organizations," the pro-Israel groups wrote in a letter send to House Foreign Affairs Committee chair Elliott Engel (D., N.Y.) and House Speaker Nancy Pelosi (D., Calif.).',
             'House Speaker Nancy Pelosi\u2019s son Paul Pelosi Jr. visited Ukraine in 2017 to meet with government officials in connection to a business initiative. Now, unearthed records reveal that Paul Pelosi Jr. was an executive of a gas industry company that did business in Ukraine – and his mother Nancy Pelosi was featured in one of the company\u2019s promotional videos. Journalist Patrick Howley broke the story Thursday: Paul Pelosi Jr. travelled to Kiev, Ukraine in July 2017 in his capacity as executive director of the Corporate Governance Initiative, a position that he accepted months earlier in February 2017. Pelosi Jr. said that he was in Ukraine to discuss a youth soccer partnership with the government. The American Mirror, which flagged Pelosi Jr.\u2019s appearance in 2017, preserved a clip of Pelosi Jr. on the Ukrainian station following the video\u2019s removal from YouTube. As Patrick Howley found, Paul Pelosi Jr. previously held top positions with the energy-sector companies Viscoil Group and NRGLab. On March 5, 2013, NRGLab New Technology posted two videos on Youtube. One video opened with a clip of Nancy Pelosi discussing energy-efficient technology, followed by a direct-to-camera statement from her son Paul Pelosi Jr., filmed in Washington, D.C. in 2010. “My name\u2019s Paul Pelosi. Of course I\u2019m on the board of Viscoil. And Viscoil is here today to talk about accelerating the future. It\u2019s about using cars in a more efficient manner. It\u2019s about utilizing natural resources, whether it be electricity, or gas, or fossil fuels in a more efficient way. And Viscoil is a part of that solution,” Paul Pelosi Jr. said in the video. “That\u2019s what Viscoil does. It utilizes technology to maximize the use of natural resources, like oil and other resources,” Pelosi Jr. said. “Paul Pelosi Jr. is a member of management team of Viscoil Group of Companies and NRGLab. Paul Pelosi Jr. is interested in developing clean energy that can replace gasoline and diesel as transportation fuel sources,” read the description on the video. Another video posted that same day confirmed that NRGLab was working in Ukraine. “Walter Afanasieff, Brandon Stone, Mika Newton and other artists are actively involved in promoting the clean technology of Viscoil Group of Companies and NRGLab. For example, Mika Newton helped to secure the rights to build a plant for the production of SH-boxes in Ukraine,” read the description of an NRGLab & Research Council video. Mika Newton is a Ukrainian-born singer. A 2013 interview with NRGLab\u2019s lead technician Zeev Drori reported that NRGLab was funded by environmental businesswoman Ana Shell, who writes about Ukraine energy issues on her blog. In the interview, Drori discussed gas generation techniques in Ukraine. “Currently, Ana Shell Fund also provide finances and support to groom talented artistes like Mika Newton, a Ukrainian singer who represented Ukraine in the Eurovision song contest in 2011,” reported a 2013 interview with Ana Shell, referring to the singer who brokered the S-H box deal in Ukraine. “In brief, SH-boxes are environmentally friendly generators which employs the use of poly-crystal technology to producing electricity from environmental heat. With the SH-boxes, electricity could be generated more efficiently and cleanly at a lower cost,” the interview with Ana Shell notes. Here is Mika Newton performing as Ukraine\u2019s representative on a Eurovision talent contest',
             'House Democrats voted Thursday to fast-track an electronic medical records system that would serve illegal immigrants, something America\u2019s veterans have been seeking for years. The House passed the bill on a largely party line vote of 230-184, American Military News reported. Only two Republican congressmen supported the bill — Brian Fitzpatrick of Pennsylvania and Fred Upton of Michigan. The proposal has yet to be considered by the Republican-controlled Senate. The Democratic proposal would require the Department of Homeland Security and U.S. Customs and Border Patrol to create a universally connected Electronic Health Records system. Part of that system\u2019s job will be to provide information to Congress on the health of migrants who enter the country illegally, according to The Hill. The bill gives the DHS 90 days after the bill receives final approval to get the job done. In contrast, the Veterans Administration has been working for years to implement a similar EHR system for veterans. According to American Military News, veterans do have access to EHRs at individual care providers and facilities, however, their records are not universally connected to follow them through all components of the DHS agency and are unable to be accessed electronically by other providers in the agency. Democratic Rep. Lauren Underwood of Illinois said it is important for Congress to ensure that workers at the border are doing their jobs. “As DHS works to improve its medical screening of children and migrants at the border to ensure there is a minimum standard of care, the need for proper record keeping on those screenings will only increase,” she said. Underwood said that the bill was based on her experiences touring America\u2019s southern Border. “When I was at the border I saw busy, overworked Border Patrol officials having to keep health records on paper. I also saw how these records don\u2019t follow migrants between facilities and transfers of custody,” she said. But critics wondered how adding more responsibilities to overworked Border Patrol officials would fix anything. “I oppose this bill because it is poorly conceived, erroneously drafted and extremely risky,” Republican Rep. Jim Banks of Indiana said. “This bill would require the Border Patrol to divert resources from its core missions and create a new medical screening for those who illegally cross and enter the country between ports of entry.” Republican Rep. Mark Walker of North Carolina said in a statement that the Democratic bill was wrong on multiple levels. “Earlier this week, I said impeachment was the House Democrats only agenda. I was mistaken. They just passed legislation that will give illegal immigrants better health care records than our veterans and servicemembers. More and more they are making their priorities clear and it\u2019s absolutely shameful,” he said. Walker noted that Democrats want illegal immigrants to have universal electronic health records faster than they are available for veterans. “The Veterans Administration (VA) will not have similar electronic health records systems in place for nine years and the Department of Defense (DOD) will not have those systems for another five years,” he said. Tennessee Republican Rep. Tim Burchett said in a Twitter video that Republicans added an amendment to the Democratic bill to ensure veterans receive the same treatment as illegal immigrants under the bill. Democrats voted down that amendment, Burchett said. While a modern EHR system is set to be implemented in some VA hospitals as soon as 2020, the full modernization effort, which officially began last May, was set to be rolled out over a decade, the Military Times reported. But it gets worse, Walker said. “Additionally, the new requirements are not funded, so DHS would be forced to take funds from other agency efforts including border security, combatting terrorism, stopping drug and human trafficking, and natural disaster response,” he said. Republican Rep. Phil Roe of Tennessee agreed. “If we cannot deliver a modern medical record for the men and women who put their lives on the line for us sooner than that, it is impossible to think that we could or should deliver one …  for those who knowingly enter our country illegally,” Roe said in a statement. “We should not treat anyone, particularly someone who breaks our laws, better than we treat our heroes. DHS staff along the border are doing the best they can to keep up with screening illegal border crossers and this bill will make it harder for them to do their job without actually helping anyone,” he said.',
             'Reminiscent of Hillary Clinton\u2019s infamous comment that Trump supporters were a “basket of deplorables,” pro-abortion former Vice President Joe Biden made another negative reference to backers of President Donald Trump. It\u2019s increasingly becoming the case that abortion activists are unable to provide a counter to President Trump and his supporters without engaging in hateful name-calling or violence. Last week, Sen. Susan Collin\u2019s office says it has received threats of rape and other violence from abortion activists who want her to vote against U.S. Supreme Court nominee Brett Kavanaugh. And a popular conservative blogger is under police protection after she received rape and death threats because of a pro-life tweet. Former Vice President Joseph R. Biden assailed President Trump\u2019s supporters during a speech Saturday at the annual Human Rights Campaign dinner in Washington, lamenting that “virulent people” and the “dregs of society” still had a friend in the White House. Mr. Biden told an enthusiastic crowd of LGBTQ rights advocates that social conservatives at home and abroad who used religion or culture as a “license to discriminate” were committing a “crime” of prejudice. “Despite losing in the courts and in the court of public opinion, these forces of intolerance remain determined to undermine and roll back the progress you all have made,” he argued. “This time they — not you — have an ally in the White House. “They\u2019re a small percentage of the American people, virulent people,” he continued. “Some of them the dregs of society. And instead of using the full might of the executive branch to secure justice, dignity [and] safety for all, the president uses the White House as a literal, literal bully pulpit, callously exerting his power over those who have little or none.”']
     }
fake_2019 = pd.DataFrame(data = d)

In [36]:
fake_2019

,title,text
0,Trump's grandfather was a pimp and tax evader;...,Most families of enormous wealth have a dark a...
1,Pelosi Diverts $2.4 Billion From Social Securi...,"Freedom isn’t free, the saying goes. And neit..."
2,Ocasio-Cortez Proposes Nationwide Motorcycle Ban,Junior Congresswoman Alexandra Ocasio-Cortez h...
3,Trump Is Now Trying To Get Mike Pence Impeached,"During a press conference, Trump said that if ..."
4,Omar Holding Secret Fundraisers With Islamic G...,"Democratic freshman Rep. Ilhan Omar (D., Minn...."
5,BREAKING: Nancy Pelosi’s Son Was Exec At Gas C...,House Speaker Nancy Pelosi’s son Paul Pelosi J...
6,Dems Vote To Enhance Med Care for Illegals Now...,House Democrats voted Thursday to fast-track a...
7,Joe Biden Calls Trump Supporters the “Dregs of...,Reminiscent of Hillary Clinton’s infamous comm...


Let's make sure these articles weren't in the training set

In [37]:
df_fake = pd.read_csv(path + 'Data/political_news/Fake.csv')
df = pd.read_csv(path + 'Data/political_news/cleaned_data.csv')

In [39]:
df_fake["title_clean"] = df_fake["title"].apply(lambda x: x.split())
df_fake["title_clean"] = df_fake["title_clean"].apply(lambda x: [item.lower() for item in x])

In [40]:
def check_title(x, check_words):
  # check if title contains all words in check_words
  count = 0
  for item in check_words:
    if item in x:
      count += 1
  if count == len(check_words):
    return True
  else:
    return False

In [41]:
# check for similar titles
fake_2019.title

0    Trump's grandfather was a pimp and tax evader;...
1    Pelosi Diverts $2.4 Billion From Social Securi...
2     Ocasio-Cortez Proposes Nationwide Motorcycle Ban
3      Trump Is Now Trying To Get Mike Pence Impeached
4    Omar Holding Secret Fundraisers With Islamic G...
5    BREAKING: Nancy Pelosi’s Son Was Exec At Gas C...
6    Dems Vote To Enhance Med Care for Illegals Now...
7    Joe Biden Calls Trump Supporters the “Dregs of...
Name: title, dtype: object

In [42]:
test_list = [['grandfather','trump'],
             ['pelosi','social','security'],
             ['motorcycle', 'ban'],
             ['pence','impeached'],
             ['omar','islamic','fundraisers'],
             ['pelosi', 'son', 'ukraine'],
             ['dem','med','illegals'],
             ['biden','trump','"dregs']]

for item in test_list:
  if True in df_fake["title_clean"].apply(check_title, args = (item,)).value_counts().index:
    print(item)
    

['pence', 'impeached']


In [43]:
# let's check on the pence impeachment article
item=['pence', 'impeached']
np.where(df_fake["title_clean"].apply(check_title, args = (item,)))

(array([1220]),)

In [44]:
df_fake.at[1220,"title_clean"] # okay, so this is a different article 

['comey',
 'might',
 'have',
 'just',
 'made',
 'sure',
 'we',
 'don’t',
 'end',
 'up',
 'with',
 'president',
 'pence',
 'when',
 'trump',
 'is',
 'impeached']

In [45]:
fake_2019.head()

,title,text
0,Trump's grandfather was a pimp and tax evader;...,Most families of enormous wealth have a dark a...
1,Pelosi Diverts $2.4 Billion From Social Securi...,"Freedom isn’t free, the saying goes. And neit..."
2,Ocasio-Cortez Proposes Nationwide Motorcycle Ban,Junior Congresswoman Alexandra Ocasio-Cortez h...
3,Trump Is Now Trying To Get Mike Pence Impeached,"During a press conference, Trump said that if ..."
4,Omar Holding Secret Fundraisers With Islamic G...,"Democratic freshman Rep. Ilhan Omar (D., Minn...."


In [46]:
# apply preprocessing step
fake_2019["text_cleaned"] = fake_2019["text"].apply(cleanText)
fake_2019["title_cleaned"] = fake_2019["title"].apply(cleanText)
# add period at end of title. 
fake_2019["title_cleaned"] = fake_2019["title_cleaned"].apply(lambda x: x +". ")
fake_2019["title_text_cleaned"] = fake_2019["title_cleaned"] + fake_2019["text_cleaned"]

In [47]:
fake_2019.head()

,title,text,text_cleaned,title_cleaned,title_text_cleaned
0,Trump's grandfather was a pimp and tax evader;...,Most families of enormous wealth have a dark a...,Most families of enormous wealth have a dark a...,Trump's grandfather was a pimp and tax evader;...,Trump's grandfather was a pimp and tax evader;...
1,Pelosi Diverts $2.4 Billion From Social Securi...,"Freedom isn’t free, the saying goes. And neit...","Freedom isn’t free, the saying goes. And neith...",Pelosi Diverts $2.4 Billion From Social Securi...,Pelosi Diverts $2.4 Billion From Social Securi...
2,Ocasio-Cortez Proposes Nationwide Motorcycle Ban,Junior Congresswoman Alexandra Ocasio-Cortez h...,Junior Congresswoman Alexandra Ocasio-Cortez h...,Ocasio-Cortez Proposes Nationwide Motorcycle B...,Ocasio-Cortez Proposes Nationwide Motorcycle B...
3,Trump Is Now Trying To Get Mike Pence Impeached,"During a press conference, Trump said that if ...","During a press conference, Trump said that if ...",Trump Is Now Trying To Get Mike Pence Impeached.,Trump Is Now Trying To Get Mike Pence Impeache...
4,Omar Holding Secret Fundraisers With Islamic G...,"Democratic freshman Rep. Ilhan Omar (D., Minn....","Democratic freshman Rep. Ilhan Omar (D., Minn....",Omar Holding Secret Fundraisers With Islamic G...,Omar Holding Secret Fundraisers With Islamic G...


In [48]:
new_new = fake_2019.title_text_cleaned.tolist()

In [49]:
new_new

['Trump\'s grandfather was a pimp and tax evader; his father a member of the KKK. Most families of enormous wealth have a dark and sometimes scandalous, even monstrous past. Donald Trump’s clan is no exception to that rule. His grandfather was a pimp and a tax evader, his father a racist who would in the course of his life, clash with New York City Police as a member of the Ku Klux Klan and then as a wealthy real estate magnate, refuse to rent to people of color. Donald Trump’s legacy is anything but a rag to riches story. His dad kicked the bucket with $250-$300 million in the bank. The man who wants to ban all people of a particular religion from travel wasn’t born with a silver spoon in his mouth, his was white gold. The only thing more obnoxious than Donald Trump himself, is his family’s money-grubbing, bigoted history. Donald Trump’s grandfather, Frederick or Friedrich Trump, made his money from operating a decadent restaurant and hotel during the Gold Rush at Klondike in the Yuko

In [52]:
max_len = 512

# tokenize and encode sequences in the training set
tokens_new = tokenizer.batch_encode_plus(
    new_new,
    max_length = max_len,
    padding='max_length',
    truncation=True
)

# convert lists to tensors
new_seq = torch.tensor(tokens_new['input_ids'])
new_mask = torch.tensor(tokens_new['attention_mask'])

with torch.no_grad():
  new_preds = model(new_seq.to(device), new_mask.to(device)) #push to gpu
  new_preds = new_preds.detach().cpu().numpy()

In [53]:
np.argmax(new_preds, axis = 1)

array([1, 1, 1, 1, 0, 0, 0, 0])